In [1]:
#
import pytz
import pandas
import datetime
from matplotlib import pyplot

#
from m_utils.transform import lag_it
from mpydge.wrap.new_data import DataHandler, MaskHandler
from mpydge.chaotic.the_new_pipe import SimplePipe
from new_insane import Insane, Neakt
from new_insane import XBR
from new_insane import MAE, R2_adj
from reporter import reported_n_muted

In [2]:

print("Process Started:\t {0}".format(datetime.datetime.utcnow().replace(tzinfo=pytz.utc)))
#
d = './result.csv'

dataset = pandas.read_csv(d, sep=';')

dataset = dataset.set_index(['time'])
# dataset = dataset.sort_index(ascending=True)

# dataset = lag_it(dataset, n_lags=1, exactly=False)
dataset = dataset.dropna()

dogtag = dataset.copy()
drops = ['id', 'title', 'news_time'] + ['ticker']  # ticker should be embedded!
dogtag = dogtag.drop(columns=drops)
cols_to_drop = [x for x in dogtag.columns.values if ('LAG0' in x and 'close' not in x)]
dogtag = dogtag.drop(columns=cols_to_drop)

Process Started:	 2020-09-20 21:59:51.649886+00:00


In [16]:

target = 'close_LAG0'
qualitative = []
quantitative = [x for x in dogtag.columns.values if 'LAG0' not in x]

X_names = [qualitative + quantitative + [target], qualitative + quantitative]
Y_names = [target, target]
outputs_0 = qualitative + quantitative + [target]
output_spec = [{x: 'float64' for x in outputs_0}, {target: 'float64'}]

g_mask = [x in quantitative and 'E_USE' not in x for x in X_names[0]]  # !
target_mask = [x == target for x in X_names[0]]

# they are similar both for target and factors, remove redundant, pls
maskeds = [{Insane(my_name='Nothing'): g_mask, Insane(my_name='Nothing'): target_mask},
           {Insane(my_name='TanhLnPct'): g_mask, Insane(my_name='TanhLnPct'): target_mask},
           {Insane(my_name='LnPct'): g_mask, Insane(my_name='LnPct'): target_mask},
           {Insane(my_name='Whiten'): g_mask, Insane(my_name='Whiten'): target_mask},
           {Insane(my_name='TanhWhiten'): g_mask, Insane(my_name='TanhWhiten'): target_mask}]

maskeds_coded = ['Nothing', 'TanhLnPct', 'LnPct', 'Whiten', 'TanhWhiten']

dogtag = dogtag[X_names[0]]

In [18]:

j = 2
nk_args = {'masked': maskeds[j], 'coded': maskeds_coded[j]}
nk = Neakt(**nk_args)

nk.fit(X=dogtag.values, Y=None)
pk = nk.predict(X=dogtag.values)

E:\venv\financial_news_re\lib\site-packages\m_utils\transformations.py:38: RuntimeWarning: divide by zero encountered in log
  self.check_row = numpy.log(data[[1], :]) - numpy.log(data[[0], :])
E:\venv\financial_news_re\lib\site-packages\m_utils\transformations.py:38: RuntimeWarning: invalid value encountered in subtract
  self.check_row = numpy.log(data[[1], :]) - numpy.log(data[[0], :])
E:\venv\financial_news_re\lib\site-packages\m_utils\transformations.py:43: RuntimeWarning: divide by zero encountered in log
  data_log_pct = numpy.log(data) - numpy.log(numpy.roll(data, shift=1, axis=0))
E:\venv\financial_news_re\lib\site-packages\m_utils\transformations.py:43: RuntimeWarning: invalid value encountered in subtract
  data_log_pct = numpy.log(data) - numpy.log(numpy.roll(data, shift=1, axis=0))


In [19]:
nk.say_my_name()

'LnPct'

In [20]:
dogtag

,lag,open_LAG1,close_LAG1,high_LAG1,low_LAG1,volume_LAG1,open_LAG2,close_LAG2,high_LAG2,low_LAG2,...,E_USE_503,E_USE_504,E_USE_505,E_USE_506,E_USE_507,E_USE_508,E_USE_509,E_USE_510,E_USE_511,close_LAG0
time,,,,,,,,,,,,,,,,,,,,,
2019-09-23 15:54:00+00:00,2.0,139.36,139.37,139.37,139.33,4975.0,139.41,139.36,139.41,139.35,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,139.42
2019-09-23 15:59:00+00:00,7.0,139.43,139.39,139.44,139.39,5904.0,139.45,139.43,139.46,139.43,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,139.38
2019-09-23 16:02:00+00:00,10.0,139.35,139.36,139.40,139.35,5249.0,139.39,139.36,139.40,139.36,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,139.35
2019-09-23 16:04:00+00:00,12.0,139.34,139.37,139.37,139.32,5789.0,139.37,139.35,139.38,139.35,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,139.42
2019-09-23 16:05:00+00:00,13.0,139.36,139.42,139.42,139.35,6275.0,139.34,139.37,139.37,139.32,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,139.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23 15:20:00+00:00,17.0,139.27,139.24,139.28,139.21,5294.0,139.24,139.27,139.28,139.22,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,139.20
2019-09-23 15:42:00+00:00,39.0,139.48,139.54,139.55,139.46,15689.0,139.48,139.47,139.55,139.47,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,139.56
2019-09-23 16:01:00+00:00,58.0,139.39,139.36,139.40,139.36,9772.0,139.38,139.38,139.40,139.37,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,139.36


In [21]:
pandas.DataFrame(pk)

,0,1,2,3,4,5,6,7,8,9,...,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,NaN
1,1.252763,0.000502,0.000143,0.000502,0.000431,0.171205,0.000287,0.000502,0.000359,0.000574,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,-0.000287
2,0.356675,-0.000574,-0.000215,-0.000287,-0.000287,-0.117593,-0.000430,-0.000502,-0.000430,-0.000502,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,-0.000215
3,0.182322,-0.000072,0.000072,-0.000215,-0.000215,0.097922,-0.000143,-0.000072,-0.000143,-0.000072,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,0.000502
4,0.080043,0.000144,0.000359,0.000359,0.000215,0.080614,-0.000215,0.000144,-0.000072,-0.000215,...,0.052154,0.038013,0.040245,0.070622,-0.062802,-0.009583,0.021909,-0.022754,0.007132,0.000789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164417,-1.016934,-0.003655,-0.003656,-0.003655,-0.003800,-1.059120,-0.003942,-0.003655,-0.003798,-0.003943,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,-0.003514
164418,0.830348,0.001507,0.002152,0.001937,0.001794,1.086386,0.001722,0.001435,0.001937,0.001794,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,0.002583
164419,0.396881,-0.000645,-0.001291,-0.001075,-0.000717,-0.473439,-0.000717,-0.000646,-0.001075,-0.000717,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,-0.001434
164420,0.358398,0.000789,0.000789,0.000789,0.000789,0.247087,0.000789,0.000789,0.000789,0.000789,...,0.063910,0.036657,0.034505,-0.066783,0.076461,-0.063666,-0.022658,0.052075,-0.025398,0.000789


In [ ]:
target_mask[-10:]